# Caso Procesamiento Imágenes - Sentimientos

El maching learning nos permite realizar aprendizajes automáticos con muchos fines. En este caso vamos a utilizar las marcas faciales y un algoritmo de machine learning para ver si podemos predecir las emociones y sentimientos de una persona a partir de su imagen.

**¿Qué objetivos tienes este caso?**

Como vimos en clase, este tipo de algoritmo es la base para el reconocimiento facial en dispositivos móviles o sistemas de reconocimiento en cajeros para detectar sentimiento como por ejemplo en robos... Tiene diferentes utilidades tanto en el sector de la ciberseguridad como en otros sectores. 

Este algoritmo, a partir de una base de datos de imágenes, detecta los puntos principales por sentimiento para, en la foto o imagen de entrada (o en la realizad en la cámara con imagen en tiempo real), detecte cual de los sentimientos pertenece a esta. En este ejemplo, para que veáis los diferentes resultados he incorporado los diferentes valores por sentimiento en la foto resultante.

**Importamos las librerías necesarias**

In [8]:
import cv2, glob, random, math, numpy as np, dlib
from sklearn.svm import SVC

Creamos nuestra lista de sentimientos. Estos son los que el algoritmo buscará en la base de datos de imágenes para realizar el estudio. 

In [9]:
emotions = ["miedo", "feliz", "neutral", "tristeza", "sorpresa", "enfado"]

Definimos el [Clahe](http://amroamroamro.github.io/mexopencv/opencv/clahe_demo_gui.html) (ecualización de histogramas). Los utilizaremos para mejorar el contraste de nuestras imágenes.

**Definimos el detector y predictor de marcas faciales**
Este programa de ejemplo muestra cómo encontrar rostros humanos frontales en una imagen y estimar su pose. La pose toma la forma de 68 puntos de referencia. Estos son puntos en la cara, como las esquinas de la boca, a lo largo de las cejas, en los ojos, etc. 

**Support vector machines (SVM)**
#Establecemos el SVM en clasificador como un vector de apoyo de máquinas polinomial kernel

In [10]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat") 
clf = SVC(kernel='linear', probability=True, tol=1e-3) 

## Definimos 3 funciones

**get_files(emotion)**  
En esta función lo que realizaresmos es seleccionar la carpeta donde se encuentran las diferentes carpetas de emociones, ordenamos la lista de ficheros con  random.shuffle() y seleccionamos la imagen en la que vamos a realizar la predicción

In [11]:
def get_files(emotion):
    files = glob.glob("dataset/%s/*" %emotion)
    random.shuffle(files)
    training = files[:int(len(files))] 
    prediction=['dataset/2.png'] #Se añade la imagen que queremos procesar
    return training, prediction

El objetivo es leer el conjunto de datos existente en un conjunto de entrenamiento y predicción con las etiquetas correspondientes, entrenar al clasificador (utilizamos máquinas de vectores de soporte con kernel lineal de SKLearn), y evaluar el resultado. Esta evaluación se realizará en dos pasos; Primero, obtenemos una precisión general después de diez vueltas de bucle de segmentación, entrenamiento y predicción de datos diferentes, y luego evaluaremos las probabilidades predictivas.
  
**get_landmarks(image)**  

In [12]:
def get_landmarks(image):
   detections = detector(image, 1)
   for k,d in enumerate(detections): #Para todas las cara detectadas de forma individual
      shape = predictor(image, d) # Dibuja puntos de referencia faciales con la clase predictor
      xlist = []
      ylist = []
      for i in range(1,68): #Guarda coordenadas X e Y en dos listas
          xlist.append(float(shape.part(i).x))
          ylist.append(float(shape.part(i).y))
 
      xmean = np.mean(xlist) #Obtiene la media de ambos ejes para determinar el centro de gravedad
      ymean = np.mean(ylist)
      xcentral = [(x-xmean) for x in xlist] #Calcula distancia entre cada punto y el punto central en ambos ejes
      ycentral = [(y-ymean) for y in ylist]

      if xlist[26] == xlist[29]: #Si la coordenada x del conjunto son las mismas, el ángulo es 0,  evitamos el error 'divide by 0' en la función
          anglenose = 0
      else:
          anglenose = int(math.atan((ylist[26]-ylist[29])/(xlist[26]-xlist[29]))*180/math.pi)

      if anglenose < 0:
          anglenose += 90
      else:
          anglenose -= 90

      #Guardamos todos los puntos de referencia      
      landmarks_vectorised = []
      for x, y, w, z in zip(xcentral, ycentral, xlist, ylist):
          landmarks_vectorised.append(x)
          landmarks_vectorised.append(y)
          meannp = np.asarray((ymean,xmean))
          coornp = np.asarray((z,w))
          dist = np.linalg.norm(coornp-meannp)
          anglerelative = (math.atan((z-ymean)/(w-xmean))*180/math.pi) - anglenose
          landmarks_vectorised.append(dist)
          landmarks_vectorised.append(anglerelative)

   if len(detections) < 1: 
       landmarks_vectorised = "error"
   return landmarks_vectorised

**make_sets()**  

In [13]:
def make_sets():
  training_data = []
  training_labels = []
  prediction_data = []
  prediction_labels = []
  training = []
  prediction = []
  for emotion in emotions:
      training, prediction = get_files(emotion) #Agrega los datos a la lista de entrenamiento y predicción, y genera etiquetas 0-7
      for item in training:
          image = cv2.imread(item) #Abrimos la imagen
          gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #Convertimos a escala de grises
          clahe_image = clahe.apply(gray)
          landmarks_vectorised = get_landmarks(clahe_image)
          if landmarks_vectorised == "error":
              pass
          else:
             training_data.append(landmarks_vectorised) #Vector de imágenes a la lista de datos de entrenamiento
             training_labels.append(emotions.index(emotion))

      for item in prediction: #Repetir el proceso anterior para el conjunto de predicciones
          image = cv2.imread(item)
          gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
          clahe_image = clahe.apply(gray)
          landmarks_vectorised = get_landmarks(clahe_image)
          if landmarks_vectorised == "error":
              pass
          else:
             prediction_data.append(landmarks_vectorised)
             prediction_labels.append(emotions.index(emotion))

  return training_data, training_labels, prediction_data, prediction_labels,

**Programa principal**  
En este apartado mostramos el resto del programa principal. En esta parte del código lo que se realiza es el bucle de repetición para generar los sets y generar las estimaciones.   
En la segunda parte obtenemos los puntos, los redondeamos y grabamos el resultado en la imagen con el resultado del procesado de la imangen.

In [ ]:
probam1 = np.zeros((5,10))
probam2 = np.zeros((1,5))

accur_lin = []

for i in range(0,10):
  print("Generando sets %s" %i) #Hace un muestreo aleatorio 80/20%
  training_data, training_labels, prediction_data, prediction_labels = make_sets()

  npar_train = np.array(training_data) #Gira el conjunto de entrenamiento en una matriz numpy para el clasificador
  npar_trainlabs = np.array(training_labels)
  print("Entrenamiento SVM linear %s" %i) #Entrenamiento SVM
  clf.fit(npar_train, training_labels)

  print("Obteniendo precisiones %s" %i) #Utilice la función score () para obtener mayor precisión
  npar_pred = np.array(prediction_data)
  pred_lin = clf.score(npar_pred, prediction_labels)
  print ("linear: ", pred_lin)
  accur_lin.append(pred_lin) #Guarda la precision en una lista
  proba=clf.predict_proba(prediction_data) #Genera las estimaciones de probabilidad
  print ("proba: ", proba)
  probam1[:,i]=proba[1,:]
  probam2=proba[1,:]+probam2


 
proba=probam2/10 
p1=round(proba[0,0],2)
p2=round(proba[0,1],2)
p3=round(proba[0,2],2)
p4=round(proba[0,3],2)
print("Valor medio lin svm: %.3f" %np.mean(accur_lin)) #Hacemos 10 ejecuciones para aumentar precision

frame=cv2.imread('dataset/2.png') #Aqui se añade la imagen que quieres procesar pero aqui solo se carga para el resultado final
#ploteamos el resultado
cv2.putText(frame, "Miedo: {}".format(p1), (10, 30),
 cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
cv2.putText(frame, "Feliz: {:.2f}".format(p2), (10, 60),
 cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
cv2.putText(frame, "Neutral: {}".format(p3), (10, 90),
 cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
cv2.putText(frame, "Triste: {:.2f}".format(p4), (10, 120),
 cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
 

 #Mostramos la imagen
cv2.imshow("Ventana de resultado", frame)
cv2.imwrite('resultado.png',frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

Generando sets 0


/Users/alejandromagdalenanino/Anaconda/anaconda3/envs/data/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in double_scalars


Entrenamiento SVM linear 0
Obteniendo precisiones 0
linear:  0.2
proba:  [[0.38901529 0.02853662 0.10621226 0.09946772 0.37676811]
 [0.38901529 0.02853662 0.10621226 0.09946772 0.37676811]
 [0.38901529 0.02853662 0.10621226 0.09946772 0.37676811]
 [0.38901529 0.02853662 0.10621226 0.09946772 0.37676811]
 [0.38901529 0.02853662 0.10621226 0.09946772 0.37676811]]
Generando sets 1


/Users/alejandromagdalenanino/Anaconda/anaconda3/envs/data/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in double_scalars


Entrenamiento SVM linear 1
Obteniendo precisiones 1
linear:  0.2
proba:  [[0.26071177 0.01655507 0.07555648 0.13030882 0.51686787]
 [0.26071177 0.01655507 0.07555648 0.13030882 0.51686787]
 [0.26071177 0.01655507 0.07555648 0.13030882 0.51686787]
 [0.26071177 0.01655507 0.07555648 0.13030882 0.51686787]
 [0.26071177 0.01655507 0.07555648 0.13030882 0.51686787]]
Generando sets 2


...

Entrenamiento SVM linear 4
Obteniendo precisiones 4
linear:  0.2
proba:  [[0.39703955 0.03221025 0.0889161  0.11699016 0.36484394]
 [0.39703955 0.03221025 0.0889161  0.11699016 0.36484394]
 [0.39703955 0.03221025 0.0889161  0.11699016 0.36484394]
 [0.39703955 0.03221025 0.0889161  0.11699016 0.36484394]
 [0.39703955 0.03221025 0.0889161  0.11699016 0.36484394]]
Generando sets 5


/Users/alejandromagdalenanino/Anaconda/anaconda3/envs/data/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in double_scalars


Entrenamiento SVM linear 5
Obteniendo precisiones 5
linear:  0.2
proba:  [[0.43535425 0.02789622 0.0913908  0.08906209 0.35629664]
 [0.43535425 0.02789622 0.0913908  0.08906209 0.35629664]
 [0.43535425 0.02789622 0.0913908  0.08906209 0.35629664]
 [0.43535425 0.02789622 0.0913908  0.08906209 0.35629664]
 [0.43535425 0.02789622 0.0913908  0.08906209 0.35629664]]
Generando sets 6


/Users/alejandromagdalenanino/Anaconda/anaconda3/envs/data/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in double_scalars


Entrenamiento SVM linear 6
Obteniendo precisiones 6
linear:  0.2
proba:  [[0.27844371 0.02294403 0.07588875 0.11285828 0.50986524]
 [0.27844371 0.02294403 0.07588875 0.11285828 0.50986524]
 [0.27844371 0.02294403 0.07588875 0.11285828 0.50986524]
 [0.27844371 0.02294403 0.07588875 0.11285828 0.50986524]
 [0.27844371 0.02294403 0.07588875 0.11285828 0.50986524]]
Generando sets 7
